In [1]:
import numpy as np

In [2]:
### Code is based on the second reference of the assignment's paper

#### USEFUL REFERENCES #######
## Initialisations, FV, VF according to https://www.ics.uci.edu/~welling/teaching/ICS279/LPCD.pdf
## FV, VF p 7, 8 --- Initialisation p 8
## parity check function: https://en.wikipedia.org/wiki/Hamming_code

In [3]:
H=np.loadtxt('H1.txt')
y=np.loadtxt('y1.txt')

In [4]:
print(H.shape)
print(y.shape)

(750, 1000)
(1000,)


In [5]:
## find the indexes of the factors with value one for a specific variable
## that indicates the existance of an edge between the variable and the factor at the bipartite graph
def get_active_factors(factor):   
    result = []
    for i in range(factor.shape[0]):
        if(factor[i] == 1):
            result.append(i)
    return result

In [6]:
def Factor_to_Variable_message(H,Message):

    FV_Message = np.zeros((H.shape))
    
    ## passing all the factor to variable messages
    
    factors_number = H.shape[0]
    variables_number = H.shape[1]
    
    ## for every factor
    for factor_id in range(factors_number):
        ## find the non zero factors for this parity check line of H
        active_factors = get_active_factors(H[factor_id,:])
        
        ## for every active factor
        for active_factor_index_1 in active_factors:
            message_product = 1
            ## for every other active factor
            for active_factor_index_2 in active_factors:
                ## that is no the same with the first one
                if(active_factor_index_1 != active_factor_index_2):
                    message_product = message_product * np.tanh(Message[factor_id,:][active_factor_index_2]/2)
                    
            FV_Message[factor_id,active_factor_index_1]=np.log((1+message_product)/(1-message_product))
            
    return FV_Message

In [7]:
def update_message(Ditribution,Message,FV_message,H):
    
    ## take all the incoming messages apart from the one we are going to send the message to
    
    factor_number,variable_number = Message.shape
    
    for variable_index in range(variable_number):
        
        ## get the active edges of the graph
        active_factors = get_active_factors(H[:,variable_index])
        
        for edge_index_1 in active_factors:
            message = 0
            for edge_index_2 in active_factors:
                if(edge_index_1!=edge_index_2):
                    message = FV_message[:,variable_index][edge_index_2]+message
                
            Message[edge_index_1,variable_index] = Ditribution[variable_index] + message
            
    return Message
        

In [8]:
def Variable_to_Factor_message(Ditribution,FV_Message):
    
    VF_message = []
    
    variables_number = FV_Message.shape[1]
    
    ## for every variable sum all the incoming factor to variable messages
    for i in range(variables_number):
        ## sum over columns --> variables are denoted with columns
        VF_message.append(Ditribution[i] + np.sum(FV_Message,axis = 0)[i])
        
    return VF_message
    

In [9]:
## compute the result of a round based on the last variable to factor message computed
## VF message contains real numbers --> convert them into 0,1
def compute_result(VF_message):
    result = []
    for bit in VF_message:
        if(bit > 0):
            result.append(1)
        else:
            result.append(0)
    return result

In [10]:
def BinarySymmetricChannel_Initialisation(y, H, p):
    
    distribution = []
    
    ## for thr Binary Symmetric Channel the probability log(P(y|x)) = (x-y)(x-y+1)log(p/(1-p))
    for i in range(len(y)):
        if(y[i]==1):
            distribution.append(np.log((1-p)/p))
        else:
            distribution.append(np.log(p/(1-p)))

    ## compute the initial message matrix, passing messages from y to H
    Message_Matrix = np.zeros((H.shape))
    
    for i in range(H.shape[0]):
        for j in range(H.shape[1]):
            Message_Matrix[i][j] = H[i][j]*distribution[j]
                
    return np.array(distribution),Message_Matrix

In [11]:
## check how many erros occur in the decoding we made
def parity_check(z):
    res = z%2
    wrong_bits = np.count_nonzero(res)
    if(wrong_bits == 0):
        return True,wrong_bits
    else:
        return False,wrong_bits  

In [12]:
def LDPC_Decoder(y, H, p=0.1, max_iter=20):
    
    # y: the received codeword matrix
    # LDPC Matrix
    # p: noise ratio
    # max_iter: maximum number of iterations of loopy belief propagation algorithm
    # distr_matrix: p(y|x_n = 0,1) matrix
    # FV_message: Factor to Variable message matrix
    # VF_message: Variable to Factor message matrix
    # M: Message Matrix
    
    Ditribution, Message = BinarySymmetricChannel_Initialisation(y,H,p)
    
    for i in range(max_iter):
        print("Iteration " + str(i))
        ## compute factor to variable message
        FV_message = Factor_to_Variable_message(H,Message)
        ## compute variable to factor message
        VF_message = Variable_to_Factor_message(Ditribution,FV_message)
        ## compute the result after the specific iterartion -- binary format
        result_of_round = compute_result(VF_message)
        
        ## a check needed here in order to determine if have computed a valid codeword
        z = H @ result_of_round # z is the syndrome
        
        ## compute th syndrome in order to detect whether is a wrong bit in the codeword
        term_flag,wrong_bits = parity_check(z)
        if(term_flag):
            print("Decoded succesfully!")
            return result_of_round
        else:
            print("Decoding failure with " + str(wrong_bits) + " wrong bits")
            if(i == max_iter - 1):
                return -1
        
        ## update the message
        Message = update_message(Ditribution,Message,FV_message,H)

In [13]:
res = LDPC_Decoder(y,H,p=0.25)
if(res == -1):
    print("Failure in decoding!")

Iteration 0
Decoding failure with 222 wrong bits
Iteration 1
Decoding failure with 158 wrong bits
Iteration 2
Decoding failure with 118 wrong bits
Iteration 3
Decoding failure with 94 wrong bits
Iteration 4
Decoding failure with 70 wrong bits
Iteration 5
Decoding failure with 52 wrong bits
Iteration 6
Decoding failure with 45 wrong bits
Iteration 7
Decoding failure with 28 wrong bits
Iteration 8
Decoding failure with 10 wrong bits
Iteration 9
Decoding failure with 3 wrong bits
Iteration 10
Decoded succesfully!


In [14]:
def recover_message(message):
    
    ### smash the recover message in words of 8 bits each
    ### each word represents a character
    ### result is a list of lists, each inner list represents a word/character
    flag = True
    start_index = 0
    result = []
    while(flag):
        ## append every word
        result.append(message[start_index:start_index + 8])
        start_index += 8
        if(start_index > 248):
            flag = False
            
            
    ## convert each word in the corresponding decimal
    dec_result = []
    for word in result:
        w = ''
        for bit in word:
            w += str(int(bit))
        dec_result.append(int(w,2))
        
    ## convert each decimal to a character based on its ASCII
    ## finally concatenate them in order to get the final message
    message = ''
    for word in dec_result:
        message += chr(word)
        
    return(message)

In [15]:
print(str(recover_message(res)))

Happy Holidays! Dmitry&David :)
